# Bayes Theorem for Predicting the Probability of an Email Being Spam

S = Spam
w = Word

$P(Spam|w_{1}, w_{2},..., w_{n}) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_{i}|Spam)$

The probability that an email consisting of the words $w_{1}, w_{2},... w_{n}$ is proportional to the probability that any given email is spam multiplied by the product of each word's probability to appear in a spam email.



In [1]:
import pandas as pd

PREDICTION = 'Prediction'

## Functions

In [2]:
def calculate_spam_proportion(training_emails):
    total = training_emails.shape[0]
    spam_emails = len(training_emails.loc[training_emails[PREDICTION] == 1])
            
    return spam_emails/total
            

In [25]:
def count_spam_vocab(spam_emails):
    total_spam_words = 0
    
    for index, row in spam_emails.iterrows():
        total_spam_words += sum(row.values[1:-1])
            
    return total_spam_words

In [4]:
def calculate_word_spamicity(w_spam_count, vocab, spam_vocab):
    alpha = 0.0001
    
    return (w_spam_count + alpha) / (spam_vocab + alpha * vocab)

In [12]:
def build_word_spamicity_dict(spam_emails, vocab, spam_vocab):
    spam_word_appearances = {}
    
    for (column_name, column_data) in spam_emails.iteritems():
        if column_name != 'Email No.' and column_name != PREDICTION:
            spam_word_appearances[column_name] = subm(column_data.values)
            
    for word in spam_word_appearances:
        spam_word_appearances[word] = calculate_word_spamicity(spam_word_appearances[word], vocab, spam_vocab)
            
    return spam_word_appearances

In [ ]:
def calculate_email_spamicities(testing_emails, word_spamicities):
    for index, row in testing_emails.iterrows():

# Model

## Step 1: Partition the data into training and test segments

20% of the data for testing, and the remaining 80% is training (i.e. the 80% training data will confirm whether the 20% testing data labels are correct).

In [6]:
df = pd.read_csv('emails.csv')
total_num_emails = df.shape[0]

end = total_num_emails//5

testing_data = df.iloc[:end]
training_data = df.iloc[end:]

testing_data


,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1029,Email 1030,10,6,4,3,5,2,45,10,0,...,0,0,0,0,0,0,0,0,0,0
1030,Email 1031,8,19,18,7,3,3,150,8,11,...,0,0,0,2,0,0,0,2,0,0
1031,Email 1032,0,0,1,0,0,2,7,0,0,...,0,0,0,0,0,0,0,0,0,0
1032,Email 1033,1,2,1,4,1,2,108,5,0,...,0,0,0,0,0,0,0,2,0,1


In [7]:
training_data

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
1034,Email 1035,18,19,18,18,3,6,143,3,0,...,0,0,0,0,0,0,0,3,0,0
1035,Email 1036,4,5,3,0,2,2,19,0,0,...,0,0,0,0,0,0,0,0,0,0
1036,Email 1037,0,1,1,1,0,0,6,1,0,...,0,0,0,0,0,0,0,0,0,1
1037,Email 1038,3,2,1,1,0,1,11,0,0,...,0,0,0,0,0,0,0,1,0,1
1038,Email 1039,11,7,11,11,16,0,98,3,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,Email 5168,2,2,2,3,0,0,32,0,0,...,0,0,0,0,0,0,0,0,0,0
5168,Email 5169,35,27,11,2,6,5,151,4,3,...,0,0,0,0,0,0,0,1,0,0
5169,Email 5170,0,0,1,1,0,0,11,0,0,...,0,0,0,0,0,0,0,0,0,1
5170,Email 5171,2,7,1,0,2,1,28,2,0,...,0,0,0,0,0,0,0,1,0,1


## Step 2: Get probability that any one email in the training data is spam

In the labelled dataset, count the number of spam and ham emails.

$P(Spam) = \frac{Spam\,Emails}{Total\,Emails}$

In [8]:
spam_proportion = calculate_spam_proportion(training_data)
spam_proportion

0.291445142580957

## Step 3: Get the "spamicity" probability of each word in the testing data email

**w** = word
<br>**vocab** = total words in dataset
<br>**spam_vocab**
<br>**wi_spam_count**

Count all unique words in the labelled dataset to get **vocab**.

Count the total number of words in labelled spam emails (ignoring uniqueness) to get **spam_vocab**.

For each word **w**, count all instances of the word in the spam emails to get **wi_spam_count**.

Calculate spamicity of each word and store the word and its spamicity in a dictionary

$P(w_{i}|Spam) = \frac{wi\_spam\_count\,+\,\alpha}{spam\_vocab\,+\,\alpha \cdot vocab}$

$\alpha$ is a coefficient that prevents a probability from being 0.


In [9]:
# Subtract 2 for "Email No." and "Prediction" columns
total_vocab = len(training_data.columns) - 2
total_vocab

3000

In [26]:
spam_training_emails = training_data.loc[training_data[PREDICTION] == 1]

total_spam_words = count_spam_vocab(spam_training_emails)
total_spam_words

1708345

In [27]:
word_spamicities = build_word_spamicity_dict(spam_training_emails, total_vocab, total_spam_words)
word_spamicities

{'the': 0.004717430428145879,
 'to': 0.00489245359237386,
 'ect': 0.0017929631088047598,
 'and': 0.002951394018527753,
 'for': 0.0021986188038214524,
 'of': 0.003282123409125778,
 'a': 0.04765020286004241,
 'you': 0.0023742273298027043,
 'hou': 0.000437850649982764,
 'in': 0.010801680491642995,
 'on': 0.008494769822002613,
 'is': 0.004925233850557027,
 'this': 0.0011502359037133768,
 'enron': 5.853617532708405e-11,
 'i': 0.04623011524660735,
 'be': 0.0025826161139671238,
 'that': 0.0006708246277845585,
 'will': 0.00032780264036784597,
 'have': 0.00046653337589303516,
 'with': 0.000984578527537729,
 'your': 0.0009722859307190414,
 'at': 0.0060333236494987275,
 'we': 0.0013761855404759214,
 's': 0.03889143494585082,
 'are': 0.001109260580984418,
 'it': 0.004741430260029983,
 'by': 0.00042146052089118047,
 'com': 0.0020294492571261793,
 'as': 0.003307879326269695,
 'from': 0.00048350886673788953,
 'gas': 0.00013346253828192697,
 'or': 0.0065747832712742555,
 'not': 0.000738726591163976,
 

## Step 4: Calculate the "spamicity" of the email

Multiply spamicities of each word together to get $\prod_{i=1}^{n}P(w_{i}|Spam)$.

Multiply that product by the probability that any email is spam.

## Step 5: Repeat steps 2-4 calculating the "hamicity" of each email

## Step 6: Label emails in test dataset and compare

A probability greater then 0.5 will indicate whether an email is ham or spam.